## Day 25 Lecture 1 Assignment

In this assignment, we will evaluate the performance of the model we built yesterday on the Chicago traffic crash data. We will also perform hyperparameter tuning and evaluate a final model using additional metrics (e.g. AUC-ROC, precision, recall, etc.)

In [ ]:
%load_ext nb_black

In [11]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.calibration import calibration_curve
from sklearn.metrics import log_loss

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
crash_data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/traffic_crashes_chicago.csv')

In [3]:
crash_data.head()

,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,...,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN
0,JC334993,7/4/2019 22:33,45,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JC370822,7/30/2019 10:22,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,DIVIDED - W/MEDIAN (NOT RAISED),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JC387098,8/10/2019 17:00,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,ONE-WAY,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JC395195,8/16/2019 16:53,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,NOT DIVIDED,NaN,...,NaN,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,JC396604,8/17/2019 16:04,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,PARKING LOT,NaN,...,NaN,1.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
crash_data['label'] = crash_data['DAMAGE'] == 'OVER $1,500'
crash_data['label'] = crash_data['label'].astype(int)

crash_data['label'].value_counts(normalize=True)

1    0.563418
0    0.436582
Name: label, dtype: float64

In [5]:
keep_cols = crash_data.isna().mean() <= 0.05
crash_sub = crash_data.loc[:, keep_cols]

# Rather than imputing, dropping is fairly low impact in terms
# of percent of observations dropped.  Rather than making up data
# the decision to drop was made.
og_nrows = crash_sub.shape[0]
crash_sub = crash_sub.dropna()
new_nrows = crash_sub.shape[0]

print(f'nrows pre drop {og_nrows}')
print(f'nrows post drop {new_nrows}')
print(f'percent rows dropped {100 * (og_nrows - new_nrows) / og_nrows:.2f}%')

crash_sub.head()

nrows pre drop 372585
nrows post drop 362480
percent rows dropped 2.71%


,RD_NO,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,ALIGNMENT,...,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,label
6,JC413474,8/30/2019 14:20,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1
7,JC414382,8/31/2019 4:35,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",FIXED OBJECT,NOT DIVIDED,"CURVE, LEVEL",...,1.0,NONINCAPACITATING INJURY,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1
8,JC413930,8/30/2019 18:30,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0
9,JC415166,8/31/2019 18:50,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,2.0,NO INDICATION OF INJURY,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1
10,JC415064,8/31/2019 13:15,30,TRAFFIC SIGNAL,UNKNOWN,CLOUDY/OVERCAST,DAYLIGHT,SIDESWIPE SAME DIRECTION,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT AND LEVEL,...,2.0,NONINCAPACITATING INJURY,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1


In [6]:
keep_cols = ['label', 'POSTED_SPEED_LIMIT', 'WEATHER_CONDITION', 'INJURIES_TOTAL', 'FIRST_CRASH_TYPE']
crash_sub = crash_sub[keep_cols]
crash_sub.head()

,label,POSTED_SPEED_LIMIT,WEATHER_CONDITION,INJURIES_TOTAL,FIRST_CRASH_TYPE
6,1,30,CLEAR,0.0,REAR END
7,1,30,CLEAR,1.0,FIXED OBJECT
8,0,30,CLEAR,0.0,TURNING
9,1,30,CLEAR,0.0,PARKED MOTOR VEHICLE
10,1,30,CLOUDY/OVERCAST,1.0,SIDESWIPE SAME DIRECTION


In [7]:
drop_cats = ['WEATHER_CONDITION_CLEAR', 'FIRST_CRASH_TYPE_REAR END']
crash_sub_dummy = pd.get_dummies(crash_sub)
crash_sub_dummy = crash_sub_dummy.drop(columns=drop_cats)
crash_sub_dummy.head()

,label,POSTED_SPEED_LIMIT,INJURIES_TOTAL,WEATHER_CONDITION_BLOWING SNOW,WEATHER_CONDITION_CLOUDY/OVERCAST,WEATHER_CONDITION_FOG/SMOKE/HAZE,WEATHER_CONDITION_FREEZING RAIN/DRIZZLE,WEATHER_CONDITION_OTHER,WEATHER_CONDITION_RAIN,WEATHER_CONDITION_SEVERE CROSS WIND GATE,...,FIRST_CRASH_TYPE_PARKED MOTOR VEHICLE,FIRST_CRASH_TYPE_PEDALCYCLIST,FIRST_CRASH_TYPE_PEDESTRIAN,FIRST_CRASH_TYPE_REAR TO FRONT,FIRST_CRASH_TYPE_REAR TO REAR,FIRST_CRASH_TYPE_REAR TO SIDE,FIRST_CRASH_TYPE_SIDESWIPE OPPOSITE DIRECTION,FIRST_CRASH_TYPE_SIDESWIPE SAME DIRECTION,FIRST_CRASH_TYPE_TRAIN,FIRST_CRASH_TYPE_TURNING
6,1,30,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,30,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,30,0.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,1,30,0.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
10,1,30,1.0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
X = crash_sub_dummy.drop(columns=['label'])
y = crash_sub_dummy['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

Since we will be building on the model we built in the last assignment, we will need to redo all of the data preparation steps up to the point of model building. These steps include creating the response, missing value imputation, and one-hot encoding our selected categorical variables. The quickest way to get going would be to open last week's assignment, make a copy, and build on it from there.

Statsmodels' implementation of logistic has certain advantages over scikit-learn's, such as clean, easy to read model summary output and statistical inference values (e.g. p-values). However, scikit-learn is preferable for model evaluation, so we will switch to the scikit-learn implementation for this exercise. 

Run logistic regression on the training set and use the resulting model to make predictions on the test set. Calculate the train and test error using logarithmic loss. How do they compare to each other?

In [10]:
model = LogisticRegression(max_iter=2500)
model.fit(X_train, y_train)

train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print(f'train score {100 * train_score:.2f}%')
print(f'test score {100 * test_score:.2f}%')

train score 58.34%
test score 58.43%


In [15]:
pred_df = X_test.copy()
pred_df['actual'] = y_test
pred_df['predicted'] = model.predict(X_test)

y_true = pred_df['actual']
y_pred = pred_df['predicted']

In [18]:
log_loss(y_true,y_pred)

14.356302916758453

Next, evaluate the performance of the same model using 10-fold CV. Use the training data and labels, and print out the mean log loss for each of the 10 CV folds, as well as the overall CV-estimated test error. How do the estimates from the individual folds compare to the result from our previous single holdout set? How much variability in the estimated test error do you see across the 10 folds?

Note: scikit-learn's *cross_val_score* function provides a simple, one-line method for doing this. However, be careful - the default score returned by this function may not be log loss!

In [0]:
# answer goes here





Scikit-learn's logistic regression function has a built-in regularization parameter, C (the larger the value of C, the smaller the degree of regularization). Use cross-validation and grid search to find an optimal value for the parameter C.

In [0]:
# answer goes here





Re-train a logistic regression model using the best value of C identified by 10-fold CV on the training data and labels. Afterwards, do the following:

- Determine the precision, recall, and F1-score of our model using a cutoff/threshold of 0.5 (hint: scikit-learn's *classification_report* function may be helpful)
- Plot or otherwise generate a confusion matrix
- Plot the ROC curve for our logistic regression model

Note: the performance of our simple logistic regression model with just four features will not be very good, but this is not entirely unexpected. There are many other features that can be incorporated into the model to improve its performance; feel free to experiment!

In [0]:
# answer goes here



